![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [41]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

# Import any additional modules and start coding below
rental = pd.read_csv('rental_info.csv')

# Converting information to the datetime format
rental['rental_date'] = pd.to_datetime(rental['rental_date'])
rental['return_date'] = pd.to_datetime(rental['return_date'])

# Calculating the renting time in days
rental['rental_length_days'] = (rental['return_date'] - rental['rental_date']).dt.days

rental.head()


,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


In [42]:
# Creating a dummy variables for deleted scenes
rental['deleted_scenes'] = np.where(rental['special_features'].str.contains("Deleted Scenes"), 1, 0)

# Creating a dummy variables for behind the scenes
rental['behind_the_scenes'] = np.where(rental['special_features'].str.contains("Behind the Scenes"), 1, 0)

rental[['special_features', 'deleted_scenes', 'behind_the_scenes']].head()


,special_features,deleted_scenes,behind_the_scenes
0,"{Trailers,""Behind the Scenes""}",0,1
1,"{Trailers,""Behind the Scenes""}",0,1
2,"{Trailers,""Behind the Scenes""}",0,1
3,"{Trailers,""Behind the Scenes""}",0,1
4,"{Trailers,""Behind the Scenes""}",0,1


In [43]:
# Creating X and y with Xthe feature matrix, and y the target variable
# Dropping those columns because the model wouldn't compute correctly with them
to_drop = ['special_features', 'rental_length_days', 'rental_date', 'return_date']
X = rental.drop(to_drop, axis=1).values
y = rental['rental_length_days'].values

# Spliting the data (we chose a 20% split for training)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)


In [44]:
from sklearn.linear_model import Lasso

#Creating a lasso model to determine the best parameters
lasso = Lasso(alpha = 0.1, random_state = 9)
lasso.fit(X_train, y_train)

#Getting its coefficients
lasso_coef = lasso.coef_

# Getting the X for lasso > 0
X_lasso_train, X_lasso_test = X_train[:, lasso_coef > 0], X_test[:, lasso_coef > 0]



In [45]:
# Creating a new model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

lr = LinearRegression()
lr.fit(X_lasso_train, y_train)
y_pred = lr.predict(X_lasso_test)
mse_lr = MSE(y_test, y_pred)
print('Test MSE {:.2f}'.format(mse_lr))


Test MSE 4.81


In [46]:
params = {'n_estimators': np.arange(1, 101, 1), 'max_depth':np.arange(1,11,1)}

rf = RandomForestRegressor()
random_search = RandomizedSearchCV(rf, param_distributions = params , cv = 5, random_state = 9)
random_search.fit(X_train, y_train)

hyper_params = random_search.best_params_

rf = RandomForestRegressor(n_estimators = hyper_params['n_estimators'], max_depth = hyper_params['max_depth'], random_state = 9)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_rf = MSE(y_test, rf_pred)


In [47]:
# Random forest seems to give the lowest MSE
print("MSE rf {:.2f}".format(mse_rf))

best_model = rf
best_mse = mse_rf

MSE rf 2.23


RandomForestRegressor(max_depth=10, n_estimators=51, random_state=9)